In [24]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.ensemble import IsolationForest

In [25]:
# Cargar el dataset

filePath = "C:/Users/raul.blanco/Documents/4 - Personales/UBA/Repositorios/IA/Trabajo-Final/solar_wind.csv"
dF = pd.read_csv(filepath_or_buffer=filePath, header=0, sep=",")

df_A = dF.loc[dF['period']=='train_a']
df_A = df_A.dropna()

In [26]:
# Calcular el número de valores a reemplazar
#features = ['bx_gse', 'by_gse', 'bz_gse', 'theta_gse', 'phi_gse', 'bx_gsm', 'by_gsm', 'bz_gsm', 'theta_gsm', 'phi_gsm', 'bt', 'density', 'speed', 'temperature']
features = ['bt']
anomaly_factor = 0.17
# Crear una copia del DataFrame para no modificar el original
df_new = df_A.copy()

for a in features:
  num_replace = int(df_A[a].size * anomaly_factor)

  # Obtener una muestra aleatoria de índices de fila para reemplazar
  replace_idx = np.random.choice(df_A.index, size=num_replace, replace=False)

  # Reemplazar los valores seleccionados con valores aleatorios entre -90 y 1050
  #new_values = np.random.uniform(-90, 1050, size=num_replace)
  new_values = np.random.uniform(0, 75, size=num_replace)
  df_new.loc[replace_idx, a] = new_values

  # Crear la columna "class" con los valores correspondientes
  df_new.loc[df_A[a] != df_new[a], 'class'] = 1
  df_new.loc[df_A[a] == df_new[a], 'class'] = 0

In [27]:
print('En el todo el dataset:')
count = (df_new['class'] == 1).sum()
count2 = (df_new['class'] != 1).sum()
print(f'El número de anomalias es {count} de {count2} valores normales')
cociente = count / count2
print(f'La relacion de anomalias es {cociente}')

En el todo el dataset:
El número de anomalias es 237074 de 1157481 valores normales
La relacion de anomalias es 0.20481891279424888


In [28]:
# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
train_data = df_new.sample(frac=0.8, random_state=0)
test_data = df_new.drop(train_data.index)

In [29]:
train_df = pd.DataFrame(train_data, columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','class'])
print('En el train set:')
count = (train_df['class'] == 1).sum()
count2 = (train_df['class'] != 1).sum()
print(f'El número de anomalias es {count} de {count2} valores normales')
cociente = count / count2
print(f'La relacion de anomalias es {cociente}')

En el train set:
El número de anomalias es 189540 de 926104 valores normales
La relacion de anomalias es 0.2046638390504738


In [30]:
test_df = pd.DataFrame(test_data, columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','class'])
print('En el test:')
count = (test_df['class'] == 1).sum()
count2 = (test_df['class'] != 1).sum()
print(f'El número de anomalias es {count} de {count2} valores normales')
cociente = count / count2
print(f'La relacion de anomalias es {cociente}')

En el test:
El número de anomalias es 47534 de 231377 valores normales
La relacion de anomalias es 0.20543960722111532


In [31]:
# Preparo el dataset en X e y y elimino columnas no necesarias
y = test_data['class']
y_full = test_data
train_data = train_data.drop(["timedelta", "period", "source", 'class'], axis=1)
test_data = test_data.drop(["timedelta", "period", "source", 'class'], axis=1)

In [32]:
# Normalizar los datos utilizando StandardScaler()
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [33]:
model = IsolationForest(n_estimators=100, contamination='auto', random_state=0)

In [34]:
model.fit(train_data)

IsolationForest(random_state=0)

In [35]:
# Utilizar el modelo para hacer predicciones en los datos de prueba
predictions = model.predict(test_data)

In [36]:
# Comparar las predicciones con los valores reales de "class" y generar una matriz de confusión para evaluar el modelo
threshold = 0.05
predictions[predictions >= threshold] = 1
predictions[predictions < threshold] = 0

In [37]:
#predictions = predictions.reshape(predictions.shape[0], predictions.shape[2])
#test_data = test_data.reshape(test_data.shape[0], test_data.shape[2])


y_full = y_full.to_numpy()
confusion_matrix = pd.crosstab(y_full[:, -1], predictions, rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)

Predicted      0       1
Actual                  
0.0        20400  210977
1.0         9501   38033


In [39]:
pd_predictions = pd.DataFrame(predictions)
column_10 = predictions#[:, 10]
is_one = column_10 ==1
# Asignar 1 a los elementos que sean True y 0 a los elementos que sean False
has_ones = np.where(is_one, 1, 0)

In [40]:
has_ones_pd = pd.DataFrame(has_ones, columns=['class'])
count = (has_ones_pd['class'] == 1).sum()
count1 = (has_ones_pd['class'] == 0).sum()
print(f'El número de anomalias predecidas es {count} de {count1} valores normales')

El número de anomalias predecidas es 249010 de 29901 valores normales


In [41]:
new_column = has_ones
# Agregar la nueva columna al final del ndarray original
predictions_class = np.hstack((predictions, new_column.reshape(-1, 1)))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)